In [37]:
import requests
from bs4 import BeautifulSoup
import time as time
import pandas as pd
import pickle
import argparse
import json
from datetime import datetime, timedelta
from functools import reduce
from math import ceil
from os import path
from time import sleep

import tweepy
from api_key import key
from requests import get, codes
from requests_oauthlib import OAuth1
from selenium import webdriver
from re import findall, IGNORECASE

ModuleNotFoundError: No module named 'api_key'

In [2]:
def techmeme_scrap(search_page, dataframe):
    """This function parses a url request from the TechMeme search and appends the results to an existing pandas dataframe."""
    for num, item in enumerate(BeautifulSoup(search_page.text, 'lxml').find_all('div', {'class':'item'})):
        try:
            in_context = item.find('div', {'class' : 'iinf'}).a['href']
            for i in range(6):
                context_url = requests.get(in_context)
                if context_url.status_code == 200:
                    break
                else:
                    print('Hand smack!')
                    time(60)
                    continue
        except:
            if num <10:
                print('Item: ', num+1, '\nCannot find "in context" link.')
            continue
        date = item.find('span', {'class' : 'idate'}).text
        original_source = item.find('a').text
        original_title = item.find('div', {'class':'ii'}).text
        extra_sources = []
        extra_titles = []
        for context_item in BeautifulSoup(context_url.text, 'lxml').find_all('div', {'class':'item'}):
            try:
                if context_item.find('span')['pml'] == in_context[-9:]:
                    for extra in context_item.find_all('div', {'class':'di'}):
                        extra_sources.append(extra.find('a').text)
                        extra_titles.append(extra.find_all('a')[1].text)
                else:
                    continue
            except:
                pass
        dataframe.loc[len(dataframe)]=[date, original_source, original_title, extra_sources, extra_titles]

def techmeme_search(start_url, dataframe):
    first_request = requests.get(start_url)
    extra_pages = int(BeautifulSoup(first_request.text, 'lxml').find('h2').text.split(' ')[-1].rstrip(':'))//10
    print('Total pages to scrap: ', extra_pages+1)
    print('Page: 1')
    print(start_url)
    techmeme_scrap(first_request, dataframe)
    for page in range(1, extra_pages+1):
        next_url = start_url[:-1] + str((page*10))
        print('Page: ', page+1)
        print(next_url)
        for i in range(6):
            next_request = requests.get(next_url)
            if next_request.status_code == 200:
                techmeme_scrap(next_request, dataframe)
                break
            else:
                print('Hand smack!')
                time(60)
                continue
                

In [3]:
Tesla_df = pd.DataFrame(columns=['date', 'original_source', 'original_title', 'extra_sources', 'extra_titles'])

In [5]:
start_url = 'https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=0'
techmeme_search(start_url, Tesla_df )

Total pages to scrap:  29
Page: 1
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=0
Page:  2
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=10
Page:  3
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=20
Page:  4
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=30
Page:  5
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=40
Page:  6
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=50
Page:  7
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=60
Page:  8
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=70
Page:  9
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=80
Page:  10
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&start=90
Page:  11
https://www.techmeme.com/search/d3results.jsp?q=title%3ATesla&wm=false&star

In [7]:
Tesla_df.shape

(278, 5)

In [8]:
Tesla_df.to_csv('Tesla.csv')

In [11]:
import tweepy
import csv
import pandas as pd

In [26]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [27]:
api = tweepy.API(auth)

In [28]:
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #initialize a list to hold all the tweepy Tweets
    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]

    #write the csv
    with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)

    pass


# if __name__ == '__main__':
# #pass in the username of the account you want to download
#     get_all_tweets("J_tsar")


In [29]:
get_all_tweets('elonmusk')

Unauthorized: 401 Unauthorized
89 - Invalid or expired token.

In [33]:
def get_all_tweets(screen_name):
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret)
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print("getting tweets before {}".format(oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print("...{} tweets downloaded so far".format(len(alltweets)))
	
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
	
	#write the csv	
	with open('{}_tweets.csv'.format(screen_name), 'w') as f:
		writer = csv.writer(f)
		writer.writerow(["id","created_at","text"])
		writer.writerows(outtweets)
		print('{}_tweets.csv was successfully created.'.format(screen_name))
	pass


if __name__ == '__main__':
	#pass in the username of the account you want to download
	get_all_tweets("J_tsar")

Unauthorized: 401 Unauthorized
89 - Invalid or expired token.

<function __main__.__authorize()>